Preamble

In [1]:
import sys
import numpy as np
from math import sqrt

# 1) Overflows and Underflows
Write a Python program to "experimentally" determine the overflow and underflow limits for double precision floating point numbers. This can be done by taking 1.0, repeatedly multiplying or dividing by a factor m, and comparing the result (or its inverse) to zero. What values of m should you choose to produce the most accurate underflow and overflow?

In [2]:
# Values based on IEEE 754. Theoretical values to compare against experimental results
actualOverflow = sys.float_info.max
actualUnderflow = sys.float_info.min

print("Actual Overflow:", actualOverflow, "Actual Underflow:", actualUnderflow)

# Calculation of Overflow
test = np.float64(1.0)
m = 2

while (1/test) > 0:
    previousTest = test
    test = np.float64(test*m)
    if (1/test) == 0:
        dataType = type(previousTest)
        print("Overflow: ", previousTest, dataType)

# Calculation of Underflow
test = np.float64(1.0)

while test > 0:
    previousTest = test
    test = np.float64(test/m)
    if test == 0:
        dataType = type(previousTest)
        print("Underflow: ", previousTest, dataType)


Actual Overflow: 1.7976931348623157e+308 Actual Underflow: 2.2250738585072014e-308
Overflow:  8.98846567431158e+307 <class 'numpy.float64'>
Underflow:  5e-324 <class 'numpy.float64'>


C:\Users\mw2680\AppData\Local\Temp\ipykernel_22808\2133824088.py:13: RuntimeWarning: overflow encountered in scalar multiply
  test = np.float64(test*m)


#### Discussion 1
Experimental and theoretical over/underflow limits do not match because the experiment allows for the usage of denormalized numbers. $m=2$ was chosen for both underflow and overflow because powers of 2 can be represented exactly in binary.

# 2) Determining Machine Precision
Machine precision $\epsilon$ is the smallest number such that $1.0+\epsilon \neq 1.0$. Write a Python program to determine $\epsilon$ for double precision floating point numbers.

In [3]:
actualMachEpsilon = sys.float_info.epsilon
print(actualMachEpsilon)

epsilon = np.float64(1.0)
while (1.0 + epsilon) != 1.0:
    previousEpsilon = epsilon
    epsilon = np.float64(epsilon/2)
print(previousEpsilon)   

2.220446049250313e-16
2.220446049250313e-16


#### Discussion 2
Experimental machine epsilon exactly matches epsilon based on IEEE 754. The while loop iterativly halves tested values of epsilon until the smallest possible value is found.

# 3) Addition & Subtraction
Write a program to calculate $f(x)= \sqrt{x^2+1}-1$ and $g(x)=\frac{x^2}{\sqrt{x^2+1}+1}$ for $x=2^{-1}, 2^{-2}, ....\, 2^{-n}$. Theoretically, $f(x)=g(x)$ but numerical results will not be equivilent for large n.  Is $f(x)$ or $g(x)$ more reliable for this calculation?

In [4]:
maxIteration = 30

for i in range(1, maxIteration):
    x = 2**(-i)
    funcF = sqrt(x**2 + 1) - 1
    funcG = x**2/(sqrt(x**2 + 1) + 1)
    print("i:", i, "x:", x, "f(x):", funcF, "g(x):", funcG)

i: 1 x: 0.5 f(x): 0.1180339887498949 g(x): 0.11803398874989485
i: 2 x: 0.25 f(x): 0.030776406404415146 g(x): 0.030776406404415133
i: 3 x: 0.125 f(x): 0.0077822185373186414 g(x): 0.0077822185373187065
i: 4 x: 0.0625 f(x): 0.001951221367587408 g(x): 0.0019512213675873353
i: 5 x: 0.03125 f(x): 0.000488162098882583 g(x): 0.0004881620988826073
i: 6 x: 0.015625 f(x): 0.00012206286282867573 g(x): 0.00012206286282875901
i: 7 x: 0.0078125 f(x): 3.0517112477923547e-05 g(x): 3.0517112477923005e-05
i: 8 x: 0.00390625 f(x): 7.629365427641588e-06 g(x): 7.629365427641587e-06
i: 9 x: 0.001953125 f(x): 1.9073468138230965e-06 g(x): 1.907346813826566e-06
i: 10 x: 0.0009765625 f(x): 4.768370445162873e-07 g(x): 4.768370445163415e-07
i: 11 x: 0.00048828125 f(x): 1.1920928244535389e-07 g(x): 1.1920928244535474e-07
i: 12 x: 0.000244140625 f(x): 2.9802321943606103e-08 g(x): 2.9802321943606116e-08
i: 13 x: 0.0001220703125 f(x): 7.450580596923828e-09 g(x): 7.4505805691682525e-09
i: 14 x: 6.103515625e-05 f(x): 1.

#### Discussion 3
For large values of $i$, $\sqrt{x^2+1} \approx 1$, so $f(x)\approx 0$. $g(x)$ does not employ subtraction, so it has a smaller error at large $i$.  However, division is a computationally expensive process.  While $g(x)$ produces a more reliable result, computational accuracy and performance must be balanced on a case-by-case basis. 

# 4) Madelung Constant
Exercise 2.9, page 74
Write a program to calculate the Madelung constant, $M$, for a three dimensional face-centered cubic crystal like NaCl. You should find $M \approx 1.747565$. How large must $L$ be to approximate $M$ to 2, 3, and 4 decimal places?

#### Hints: 
<ol> <li> The Madelung constant is a purely geometric factor (i.e., do not include $\frac{e}{4\pi\epsilon_0 a}$ in the program).</li> 
<li> In the calculation you sum over a set of plus and minus charges in a large cubic box of side L. For the sum to converge correctly to M , the total charge within this box must be zero.</li>
</ol>

In [38]:
def madelung(numAtoms):
    voltage = 0
    for i in range(-numAtoms, numAtoms):
        for j in range(-numAtoms, numAtoms):
            for k in range(-numAtoms, numAtoms):
                if i != 0 or j != 0 or k != 0:  # Exclude the origin
                    dist = sqrt(i**2 + j**2 + k**2)
                    if (i + j + k) % 2 == 0:
                        # Sodium atoms are positive
                        voltContribution = 1 / dist
                    else:
                        # Chlorine atoms are negative
                        voltContribution = -1 / dist

                    voltage += voltContribution
    madelungConst = abs(voltage)
    return madelungConst

print(madelung(1))
print(madelung(2))
print(madelung(4))
print(madelung(8))
print(madelung(300))

1.4560299256299833
1.7517691333369405
1.7477211096310816
1.747574381725534
1.74756459465029


#### Discussion 4
A single atom cannot be used to approximate the Madelung constant to any relevant precision, but only 2 atoms are required to approximate the Madelung constant to 3 decimal places (1.75). 4 decimal places requires 8 atoms to be considered (1.7476).  The true Madelung constant assumes the limit where $L\rightarrow\infty$.  To simulate this, a large $L$ was chosen. With $L=300$, the program takes 54.2 seconds to run on my hardware.